You need to restart runtime after the installation otherwise it will not work.

In [1]:
!pip3 install pytextrank
!spacy download en_core_web_sm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.0/296.0 KB 25.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import spacy
from transformers.models.auto.processing_auto import AutoTokenizer
import torch
import pytextrank
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
import textwrap

tokenizer = AutoTokenizer.from_pretrained('t5-base')

model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/AIN311Project/Models/t5-base-question-generation_custom_20")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print ("device ",device)
model = model.to(device)

def keywordExt(text):
  # load a spaCy model, depending on language, scale, etc.
  nlp = spacy.load("en_core_web_sm")
  # add PyTextRank to the spaCy pipeline
  nlp.add_pipe("textrank")
  doc = nlp(text)
  # examine the top-ranked phrases in the document
  return doc._.phrases[:5]


def GenerateQuestion(text):
  encoding = tokenizer.encode_plus(text, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  output = greedy_decoding(input_ids,attention_masks)
  print (" \n ",output)

def greedy_decoding (inp_ids,attn_mask):
    greedy_output = model.generate(input_ids=inp_ids, attention_mask=attn_mask, max_length=256)
    Question =  tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
    return Question.strip().capitalize()

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [21]:
#@title Lets Generate Question
#@markdown Write the context in which you want AI to produce a question, if you want a specific question, also indicate the answer.

context = "There is only one god, and his name is Death. And there is only one thing we say to God of Death: 'Not today.'"  #@param
answer = '' #@param
#@markdown ---

if answer == '':
  keywords = keywordExt(context)
  answers = []
  for i in keywords:
    answers.append(i.text)

  lines = textwrap.wrap("\n%s\n" % context, width=100)
  print("\n".join(lines))
  print()
  print("Generated Questions: \n")
  for i in answers:
    print(i)
    text = "answer : " + i + "  " + "context : " + context
    GenerateQuestion(text)
    print()
else:
    text = "answer : " + answer + "  " + "context : " + context
    GenerateQuestion(text)
    print()

 There is only one god, and his name is Death. And there is only one thing we say to God of Death:
'Not today.'

Generated Questions: 

today
 
  What do we say to god of death?

Death
 
  What is the name of the god who created us?

God
 
  Who is the god of death?

his name
 
  What is the name of the god who is death?

only one god
 
  What is the name of the god that dies?

